In [1]:
import cv2
import PIL
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import Video, widgets
from IPython.display import display

%matplotlib inline

In [2]:
def resize_img(img, div=3):
    return cv2.resize(img, (img.shape[1]//div, img.shape[0]//div))

In [3]:
video_capture = cv2.VideoCapture('Chess_Data\\chess_easy1.mp4')

In [8]:
video_capture = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')

def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

def get_harris(chessboard):
    gray = cv2.cvtColor(chessboard, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())
    return corners, chessboard.copy()

def get_closest_corners(corners, chessboard):
    image_corners = np.array([[0, 0], 
                              [0, chessboard.shape[1]], 
                              [chessboard.shape[0], 0], 
                              [chessboard.shape[0], chessboard.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)
    return closest_corners


ret, prev_frame = video_capture.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
ECHO_LENGTH = 40
COUNTOUR_THRESHOLD = 5
AREA_THRESHOLD = 30
echo = False
curr_echo = 0

while True:   
    ret, chessboard = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(chessboard, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    frame_diff = cv2.absdiff(prev_gray, gray)
    _, thresholded_diff = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    num_contours = sum(1 for contour in contours if cv2.contourArea(contour) > AREA_THRESHOLD)
    if num_contours > COUNTOUR_THRESHOLD or echo == True: # hand detected
        chessboard = prev_frame
        curr_echo+=1
        if curr_echo>ECHO_LENGTH:
            curr_echo = 0
            echo = False
        
    else: # hand not detected
        prev_frame = chessboard
        echo = True

    corners, frame_with_boxes = get_harris(chessboard)
    closest_corners = get_closest_corners(corners, chessboard)
    prev_gray = gray.copy()
    target_square_size = 1000
    target_points = np.array([[0, 0], 
                              [0, target_square_size - 1], 
                              [target_square_size - 1, 0], 
                              [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    
    homography_matrix, _ = cv2.findHomography(closest_corners[:,::-1], target_points)
    calibrated_image = cv2.warpPerspective(chessboard, homography_matrix, (target_square_size, target_square_size))
    offset = 60
    clean_img = calibrated_image.copy()

    for x in np.linspace(offset, 1000-offset, 9, dtype=int):
        cv2.line(clean_img, (x, 0+offset), (x, 1000-offset), color=(0, 0, 255), thickness=2)  
        cv2.line(clean_img, (0+offset, x), (1000-offset, x), color=(0, 0, 255), thickness=2)
  
    cv2.imshow('Frame with Giant Bounding Box', clean_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 